In [1]:
import pandas as pd
import numpy as np
from numpy import nan as NaN
from sklearn.impute import KNNImputer
# I
#Importing files
covid = pd.read_excel("./DATA/covid19.xlsx")
temperature = pd.read_excel("./DATA/temperature.xls", sheet_name="Country_temperatureCRU")
population_density = pd.read_excel("./DATA/population_density.xls", sheet_name='Data')
population_65 = pd.read_excel("./DATA/population_65.xls", sheet_name='Data')
pkb = pd.read_excel("./DATA/pkb.xls", sheet_name='Data')
medics = pd.read_excel("./DATA/medics.xls", sheet_name='Data')
beds = pd.read_excel("./DATA/beds.xls", sheet_name='Data')
air_pollution = pd.read_excel("./DATA/air_pollution.xls", sheet_name='Data')
accidents = pd.read_excel("./DATA/accidents.xls", sheet_name='Data')

In [2]:
# STANDARIZATION OF COLUM NAMES
temperature = temperature.rename(columns={'ISO_3DIGIT':'country_id', 'Sept_temp':'temp_sep', 'Oct_temp':'temp_oct'})
pkb = pkb.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 63' : 'pkb'})
beds = beds.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 55' : 'beds'})
medics = medics.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 61' : 'medics'})
population_65 = population_65.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 63' : 'above_65'})
population_density = population_density.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 62' : 'population_density'})
accidents = accidents.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 60' : 'accidents'})
air_pollution = air_pollution.rename(columns={'World Development Indicators' : 'country_id','Unnamed: 61' : 'air_pollution'})

In [3]:
    # CASES, DEATHS
# Choosing Data from September and October
covid_from_september = covid[(covid["year"] == 2020) & (covid["month"] >= 9)]
covid_from_october = covid_from_september[(covid_from_september["month"] >= 10)]

# Sum Cases and Deaths
deaths_and_cases_from_september = covid_from_september.groupby("countriesAndTerritories").sum()
deaths_and_cases_from_september = deaths_and_cases_from_september[["deaths", "cases"]]
deaths_and_cases_from_september.rename(columns={"deaths" : "deaths_from_sep", "cases" : "cases_from_sep"}, inplace=True)
deaths_and_cases_from_september.reset_index(inplace=True)

deaths_and_cases_from_october = covid_from_october.groupby("countriesAndTerritories").sum()
deaths_and_cases_from_october = deaths_and_cases_from_october[["deaths", "cases"]]
deaths_and_cases_from_october.rename(columns={"deaths" : "deaths_from_oct", "cases" : "cases_from_oct"}, inplace=True)
deaths_and_cases_from_october.reset_index(inplace=True)

In [4]:
# PREPARE THE FINAL DATA FRAME
covid_data = covid[["countriesAndTerritories", "geoId", "countryterritoryCode", "continentExp", "popData2019"]]
covid_data = covid_data.drop_duplicates().reset_index(drop=True)
    # Removing empty row
covid_data.drop(37, axis=0, inplace=True)
covid_data.reset_index(drop=True, inplace=True)
covid_data.rename(columns = 
{"countriesAndTerritories" : "country", "geoId" : "geo_id", "countryterritoryCode" : "country_id", "continentExp" : "continent", "popData2019" : "population" }, inplace=True)



In [5]:
# NORMALIZED CASES AND DEATHS
death_and_cases_from_september_per_100k = deaths_and_cases_from_september[["deaths_from_sep", "cases_from_sep"]].multiply(100000)
death_and_cases_from_september_per_100k = death_and_cases_from_september_per_100k[["deaths_from_sep", "cases_from_sep"]].div(covid_data["population"], axis=0)
death_and_cases_from_september_per_100k.rename(columns={"deaths_from_sep":"deaths_from_sep_per_100k", "cases_from_sep":"cases_from_sep_per_100k"}, inplace=True)


incident_rate = deaths_and_cases_from_october
incident_rate.rename(columns={"deaths_from_oct":"incident_rate_deaths", "cases_from_oct":"incident_rate_cases"}, inplace=True)
incident_rate["incident_rate_deaths"] = incident_rate["incident_rate_deaths"].div(deaths_and_cases_from_september["deaths_from_sep"]).multiply(100)
incident_rate["incident_rate_cases"] = incident_rate["incident_rate_cases"].div(deaths_and_cases_from_september["cases_from_sep"]).multiply(100)
incident_rate = incident_rate.drop(["countriesAndTerritories"], axis=1)


In [6]:
# APPEND TO FINAL DATA FRAME
covid_data = covid_data.join(death_and_cases_from_september_per_100k)
covid_data = covid_data.join(incident_rate)

In [7]:
def search_by_country_id(countries_df, file_df, mod_df, col_entry):
    for code in countries_df:
        location = file_df.loc[file_df["country_id"] == code, ["country_id", col_entry]]
        if location.empty == True:
            mod_df = mod_df.append(pd.Series([code, NaN], index=["country_id", col_entry]), ignore_index = True)
        else:
            mod_df = mod_df.append(location)
    return mod_df[col_entry].to_numpy()

In [8]:
# ADD TEMPERATURE
temp_data = pd.DataFrame(columns=["country_id", "temp_sep", "temp_oct"])
temp_sep = search_by_country_id(covid_data["country_id"], temperature, temp_data, "temp_sep")
temp_oct = search_by_country_id(covid_data["country_id"], temperature, temp_data, "temp_oct")
temp_data = {
    "temp_sep" : temp_sep,
    "temp_oct" : temp_oct
}
temp_data = pd.DataFrame(temp_data)
covid_data = covid_data.join(temp_data)

In [9]:
# ADD POPULATION DENSITY
population_density_data = pd.DataFrame(columns=["country_id", "population_density"])
pop_den = search_by_country_id(covid_data["country_id"], population_density, population_density_data, "population_density")
population_density_data = {
    "population_density": pop_den
}
population_density_data = pd.DataFrame(population_density_data)
covid_data = covid_data.join(population_density_data)


In [10]:
# ADD POPULATION ABOVE 65
population_65_data = pd.DataFrame(columns=["country_id", "above_65"])
pop_65 = search_by_country_id(covid_data["country_id"], population_65, population_65_data, "above_65")
population_65_data = {
    "above_65": pop_65
}
population_65_data = pd.DataFrame(population_65_data)
covid_data = covid_data.join(population_65_data)

In [11]:
# ADD PKB
pkb_data = pd.DataFrame(columns=["country_id", "pkb"])
pkb_o = search_by_country_id(covid_data["country_id"], pkb, pkb_data, "pkb")
pkb_data = {
    "pkb" : pkb_o
}
pkb_data = pd.DataFrame(pkb_data)
covid_data = covid_data.join(pkb_data)

In [12]:
# MEDICS
medics_data = pd.DataFrame(columns=["country_id", "medics"])
med = search_by_country_id(covid_data["country_id"], medics, medics_data, "medics")
medics_data = {
    "medics" : med
}
medics_data = pd.DataFrame(medics_data)
covid_data = covid_data.join(medics_data)

In [13]:
# BEDS
beds_data = pd.DataFrame(columns=["country_id", "beds"])
beds_o = search_by_country_id(covid_data["country_id"], beds, beds_data, "beds")
beds_data = {
    "beds" : beds_o
}
beds_data = pd.DataFrame(beds_data)
covid_data = covid_data.join(beds_data)

In [14]:
# AIR POLLUTION 
air_pollution_data = pd.DataFrame(columns=["country_id", "air_pollution"])
air_pol = search_by_country_id(covid_data["country_id"], air_pollution, air_pollution_data, "air_pollution")
air_pollution_data = {
    "air_pollution" : air_pol
}
air_pollution_data = pd.DataFrame(air_pollution_data)
covid_data = covid_data.join(air_pollution_data)

In [15]:
# ACCIDENTS
accidents_data = pd.DataFrame(columns=["country_id", "accidents"])
acc = search_by_country_id(covid_data["country_id"], accidents, accidents_data, "accidents")
accidents_data = {
    "accidents" : acc
}
accidents_data = pd.DataFrame(accidents_data)
covid_data = covid_data.join(accidents_data)

In [16]:
# II
# BASIC STATISTICS
missing_values = covid_data.isnull().sum()
missing_values.name = 'missing_values'
data_stat = pd.DataFrame(missing_values)

max_values = covid_data.max(numeric_only=True)
max_values.name = 'max_values'
max_values = pd.DataFrame(max_values)
data_stat = data_stat.join(max_values)

min_values = covid_data.min(numeric_only=True)
min_values.name = 'min_values'
min_values = pd.DataFrame(min_values)
data_stat = data_stat.join(min_values)

mean_values = covid_data.mean(skipna=True, numeric_only=True)
mean_values.name = "mean_values"
mean_values = pd.DataFrame(mean_values)
data_stat = data_stat.join(mean_values)

median_values = covid_data.median(numeric_only=True)
median_values.name = "median_values"
median_values = pd.DataFrame(median_values)
data_stat = data_stat.join(median_values)

std_deviation_values = covid_data.std(numeric_only=True)
std_deviation_values.name = "std_deviation_values"
std_deviation_values = pd.DataFrame(std_deviation_values)
data_stat = data_stat.join(std_deviation_values)


In [21]:
#  k-Nearest Neighbour
covid_data_kkn = covid_data.iloc[0::,4::]
column_names = covid_data_kkn.columns
covid_data_kkn = covid_data_kkn.to_numpy()
imputer = KNNImputer(n_neighbors=10)
covid_data_kkn = imputer.fit_transform(covid_data_kkn)
covid_data_kkn = pd.DataFrame(covid_data_kkn)
covid_data_kkn.columns = column_names
print(covid_data_kkn)
covid_data_kkn = covid_data.iloc[0::, 0:4].join(covid_data_kkn)

     population  deaths_from_sep_per_100k  cases_from_sep_per_100k  \
0    38041757.0                  0.565168                13.348490   
1     2862427.0                 11.598549               623.701495   
2    43053054.0                  1.481893                52.662931   
3       76177.0                 28.880108              6039.880804   
4    31825299.0                  0.659852                33.319404   
..          ...                       ...                      ...   
208   2274625.0                  0.910337               417.808543   
209    582458.0                  0.000000                 0.000000   
210  29161922.0                  0.133736                 0.408066   
211  17861034.0                  0.352723                28.397012   
212  14645473.0                  0.416511                16.209787   

     incident_rate_deaths  incident_rate_cases   temp_sep   temp_oct  \
0               73.953488            78.495471  19.034575  12.992646   
1              

In [18]:
std_dev = data_stat['std_deviation_values']
mean_values = data_stat['mean_values']
sigma = mean_values + (std_dev * 3)
sigma = pd.DataFrame(sigma).dropna(how="all")
covid_data_sub = covid_data.iloc[0::,4::]
covid_data_sub_col = covid_data_sub.columns.tolist()

for column in covid_data_sub_col:
    for item in range(0, covid_data_sub.shape[0]):
        if covid_data_sub[column][item] <= sigma[0][column]:
            covid_data_sub[column][item] = NaN

covid_data_sub = covid_data_sub.dropna(how="all")
